In [1]:
!pip install transformers datasets evaluate
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [38]:
!pip install imblearn

In [85]:
import pandas as pd

In [86]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import evaluate

In [87]:
df = pd.read_csv('/content/Sentimental analysis data.csv')

In [88]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

# Load your dataset
file_path = "/content/Sentimental analysis data.csv"
df = pd.read_csv(file_path)

# Check the distribution of the 'status' column
print(df['status'].value_counts())

# Separate features (X) and labels (y)
X = df['statement']  # Assuming 'statement' is the text column
y = df['status']     # Assuming 'status' is the label column

# Convert X to DataFrame to work with RandomOverSampler
X = X.values.reshape(-1, 1)

# Initialize the RandomOverSampler
oversampler = RandomOverSampler(random_state=42)

# Perform oversampling
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Convert resampled data back to DataFrame for easy manipulation
df_resampled = pd.DataFrame({'statement': X_resampled.flatten(), 'status': y_resampled})

# Check the distribution after resampling
print(df_resampled['status'].value_counts())

# Save the balanced dataset (optional)
df_new = df_resampled.to_csv('balanced_sentimental_analysis_data.csv', index=False)


status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64
status
Anxiety                 16351
Normal                  16351
Depression              16351
Suicidal                16351
Stress                  16351
Bipolar                 16351
Personality disorder    16351
Name: count, dtype: int64


In [94]:
df = pd.read_csv('/content/balanced_sentimental_analysis_data.csv')
df

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...
114452,I am diagnosed with Major Depressive Disorder ...,Suicidal
114453,it has been seven months since i had to say go...,Suicidal
114454,I am currently going to university but I am la...,Suicidal
114455,I experience breakdowns everyday. Even when in...,Suicidal


In [96]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'status' is the label column
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['status'])

# Print the encoded labels to check
print(df[['status', 'labels']])


          status  labels
0        Anxiety       0
1        Anxiety       0
2        Anxiety       0
3        Anxiety       0
4        Anxiety       0
...          ...     ...
114452  Suicidal       6
114453  Suicidal       6
114454  Suicidal       6
114455  Suicidal       6
114456  Suicidal       6

[114457 rows x 2 columns]


In [97]:
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize the input text
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Return tokenized inputs and labels
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Split the data into training and validation sets
train_texts = df['statement'].values
train_labels = df['labels'].values

In [98]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [99]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=7)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [100]:
# Prepare the dataset
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)

In [101]:
# Define metrics
metric = evaluate.load("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [102]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [103]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Add validation dataset if available
    compute_metrics=compute_metrics
)

In [ ]:
try:
    # Train the model
    trainer.train()
except Exception as e:
    print(f"Error during training: {e}")

Epoch,Training Loss,Validation Loss,Accuracy
1,0.280100,0.200597,0.923360
2,0.191400,0.124552,0.954166


In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

In [30]:
# Check the distribution of the labels
print(df['status'].value_counts())

status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64


In [ ]:
# Define the labels corresponding to the output classes
labels = ['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar', 'Personality disorder']

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    predicted_label_idx = torch.argmax(probs, dim=-1).item()
    predicted_label = labels[predicted_label_idx]

    # Print probabilities for each label
    for i, prob in enumerate(probs[0]):
        print(f"{labels[i]}: {prob.item() * 100:.2f}%")

    # Return the predicted label
    return predicted_label

# Example inference
print(predict_sentiment("what a bad day"))
